In [ ]:
# from google.colab import drive
# drive.mount('/content/drive', force_remount=True)


# resturcture the train_data folder 



In [ ]:
# 

# import os
# import shutil

# # Set the path to the original folder
# orig_path = '/content/drive/Shareddrives/cse 455/cat_dog_train_data'

# # Set the path to the new folder structure
# new_path = '/content/drive/Shareddrives/cse 455/dog_cat_data_restructured'

# # Create the train, validation, and test folders
# os.makedirs(os.path.join(new_path, 'train', 'cat'))
# os.makedirs(os.path.join(new_path, 'train', 'dog'))
# os.makedirs(os.path.join(new_path, 'validation', 'cat'))
# os.makedirs(os.path.join(new_path, 'validation', 'dog'))
# os.makedirs(os.path.join(new_path, 'test', 'cat'))
# os.makedirs(os.path.join(new_path, 'test', 'dog'))

# # Get the list of files in the original cat and dog folders
# cat_files = os.listdir(os.path.join(orig_path, 'cat'))
# dog_files = os.listdir(os.path.join(orig_path, 'dog'))

# # Copy the first 1000 cat and dog files to the validation folder
# for i in range(1000):
#     shutil.copy(os.path.join(orig_path, 'cat', cat_files[i]), os.path.join(new_path, 'validation', 'cat'))
#     shutil.copy(os.path.join(orig_path, 'dog', dog_files[i]), os.path.join(new_path, 'validation', 'dog'))

# # Copy the last 1000 cat and dog files to the test folder
# for i in range(-1000, 0):
#     shutil.copy(os.path.join(orig_path, 'cat', cat_files[i]), os.path.join(new_path, 'test', 'cat'))
#     shutil.copy(os.path.join(orig_path, 'dog', dog_files[i]), os.path.join(new_path, 'test', 'dog'))

# # Copy the remaining files to the train folder
# for file in cat_files[1000:] + dog_files[1000:]:
#     if file in cat_files:
#         shutil.copy(os.path.join(orig_path, 'cat', file), os.path.join(new_path, 'train', 'cat'))
#     else:
#         shutil.copy(os.path.join(orig_path, 'dog', file), os.path.join(new_path, 'train', 'dog'))


# Train 

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import torch
import torchvision
import torch.nn as nn # All neural network modules, nn.Linear, nn.Conv2d, BatchNorm, Loss functions
import torchvision.datasets as datasets # Has standard datasets we can import in a nice way
import torchvision.transforms as transforms # Transformations we can perform on our dataset
import torch.nn.functional as F # All functions that don't have any parameters
from torch.utils.data import DataLoader, Dataset # Gives easier dataset managment and creates mini batches
from torchvision.datasets import ImageFolder
import torch.optim as optim # For all Optimization algorithms, SGD, Adam, etc.
import torch.nn.functional as F
from PIL import Image

Set Device

In [ ]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu") # use gpu or cpu


In [ ]:
DEVICE

device(type='cuda')

# Load data

In [ ]:
FAST_RUN = False
IMAGE_WIDTH=224
IMAGE_HEIGHT=224
IMAGE_SIZE=(IMAGE_WIDTH, IMAGE_HEIGHT)
IMAGE_CHANNELS=3
DATA_PATH = "/content/drive/Shareddrives/cse 455/dog_cat_data_restructured/"
BATCH_SIZE=16
SAVE_TO = "/content/drive/Shareddrives/cse 455/CatDogClassifer_models/"

In [ ]:
## add data augmentation
transform_train = transforms.Compose([
transforms.Resize((IMAGE_WIDTH, IMAGE_HEIGHT)),
transforms.RandomCrop(IMAGE_WIDTH, padding=8, padding_mode='edge'), # Take IMAGE_WIDTHxIMAGE_WIDTH crops from padded images
transforms.RandomHorizontalFlip(),    # 50% of time flip image along y-axis
transforms.ToTensor(),
transforms.Normalize(mean = (0.485, 0.456, 0.406), std = (0.229, 0.224, 0.225))
])


transform_test = transforms.Compose([
transforms.Resize((IMAGE_WIDTH, IMAGE_HEIGHT)),
transforms.ToTensor(),
transforms.Normalize(mean = (0.485, 0.456, 0.406), std = (0.229, 0.224, 0.225))
])


In [ ]:
# rm -rf "/content/drive/Shareddrives/cse 455/dog_cat_data_restructured/validation/.ipynb_checkpoints"

In [ ]:
data_train = datasets.ImageFolder(DATA_PATH + "train", transform=transform_train)
data_test = datasets.ImageFolder(DATA_PATH + "test", transform=transform_test)
data_val = datasets.ImageFolder(DATA_PATH + "validation", transform=transform_test)

# dataset_subset = torch.utils.data.Subset(data, np.random.choice(len(data), 3000, replace=False))


In [ ]:
len(data_train)
len(data_test)
len(data_val)


2001

In [ ]:
# train_size = int(0.8 * len(data))
# test_size = int(0.1 * len(data))
# val_size = len(data) - train_size - test_size

# # train_size = 2500
# # test_size = 250
# # val_size = 250
# train_size, val_size, test_size


In [ ]:
# train_set, val_set, test_set = torch.utils.data.random_split(data, [train_size, val_size, test_size])
# train_set, val_set, test_set = torch.utils.data.random_split(dataset_subset, [train_size, val_size, test_size])


In [ ]:
from collections import Counter
print(dict(Counter(data_train.targets)))

{0: 29, 1: 11175, 2: 11500}


In [ ]:
train_loader = DataLoader(
data_train,
batch_size=BATCH_SIZE,
shuffle=True
)

val_loader = DataLoader(
data_val,
batch_size=BATCH_SIZE,
shuffle=False
)

test_loader = DataLoader(
data_test,
batch_size=BATCH_SIZE,
shuffle=False

)

# Define Model

In [ ]:
class CatDogCNN(nn.Module):
    def __init__(self):
        super().__init__()

        # Convolutional layers
        self.conv1 = nn.Conv2d(in_channels = 3, out_channels = 16, kernel_size=(5, 5), stride=2, padding=1) # size 112
        self.conv2 = nn.Conv2d(in_channels = 16, out_channels = 32, kernel_size=(5, 5), stride=2, padding=1)  # size 
        self.conv3 = nn.Conv2d(in_channels = 32, out_channels = 64, kernel_size=(3, 3), padding=1)


        # batch nomarlization layers 
        self.bn1 = nn.BatchNorm2d(16)
        self.bn2 = nn.BatchNorm2d(32)
        self.bn3 = nn.BatchNorm2d(64)

        # conected layers
        # 64 * 6 * 6 = 2304 neurons
        self.fc1 = nn.Linear(in_features= 2304, out_features=500)
        self.fc2 = nn.Linear(in_features=500, out_features=50)
        self.fc3 = nn.Linear(in_features=50, out_features=2)


    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.max_pool2d(x, 2)

        x = F.relu(self.bn2(self.conv2(x)))
        x = F.max_pool2d(x, 2)

        x = F.relu(self.bn3(self.conv3(x)))
        x = F.max_pool2d(x, 2)

        # flatten
        x = x.view(x.shape[0], -1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)

        return x 

In [ ]:
class Darknet64DogCat(nn.Module):
    def __init__(self):
        super(Darknet64DogCat, self).__init__() # https://pytorch.org/docs/stable/generated/torch.nn.Conv2d.html
        self.conv1 = nn.Conv2d(3, 16, 3, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(16)

        self.conv2 = nn.Conv2d(16, 32, 3, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(32)

        self.conv3 = nn.Conv2d(32, 64, 3, padding=1, bias=False)
        self.bn3 = nn.BatchNorm2d(64)

        self.conv4 = nn.Conv2d(64, 128, 3, padding=1, bias=False)
        self.bn4 = nn.BatchNorm2d(128)

        self.conv5 = nn.Conv2d(128, 256, 3, padding=1, bias=False)
        self.bn5 = nn.BatchNorm2d(256)

        self.fc2 = nn.Linear(256, 2)

    def forward(self, x):
        x = F.max_pool2d(F.relu(self.bn1(self.conv1(x))), kernel_size=2, stride=2)
        x = F.max_pool2d(F.relu(self.bn2(self.conv2(x))), kernel_size=2, stride=2)
        x = F.max_pool2d(F.relu(self.bn3(self.conv3(x))), kernel_size=2, stride=2)
        x = F.max_pool2d(F.relu(self.bn4(self.conv4(x))), kernel_size=2, stride=2)
        x = F.max_pool2d(F.relu(self.bn5(self.conv5(x))), kernel_size=2, stride=2)
        x = F.adaptive_avg_pool2d(x, 1)
        x = torch.flatten(x, 1)
        x = self.fc2(x)
        return x

In [ ]:
model = CatDogCNN().to(DEVICE)
model

In [ ]:

from tqdm import tqdm

EPOCHS = 10

# add L2 regularization with weight decay
def train(model = model, train_loader = train_loader, val_loader = val_loader, 
          epochs=EPOCHS, lr=0.001, schedule={}, decay=0.0005):
  loss = nn.CrossEntropyLoss()
  train_losses = []
  train_accuracies = []
  val_losses = []
  val_accuracies = []
  optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=decay)


  for e in range(EPOCHS):

    train_loss = 0.0
    train_acc = 0.0
    model.train()


     # Update learning rate when scheduled
    if e in schedule:
      print ("Learning rate: %f"% schedule[e])
      for g in optimizer.param_groups:
        g['lr'] = schedule[e]


    loop = tqdm(enumerate(train_loader), total = len(train_loader))
    # train model
    for batch_idx, (x_batch, labels) in loop:
      x_batch, labels = x_batch.to(DEVICE), labels.to(DEVICE)
      # zero the parameter gradients
      optimizer.zero_grad()

       # forward + backward + optimize 
      labels_pred = model(x_batch)
      batch_loss = loss(labels_pred, labels)
      train_loss = train_loss + batch_loss.item()
      labels_pred_max = torch.argmax(labels_pred, 1)
      batch_acc = torch.sum(labels_pred_max == labels)
      train_acc = train_acc + batch_acc.item()
      batch_loss.backward()
      optimizer.step()

      # update tqdm
      loop.set_postfix(batch_idx= batch_idx, batch_acc= batch_acc.item() /BATCH_SIZE, loss= batch_loss.item())

    # Validation loop; use .no_grad() context manager to save memory.
    train_losses.append(train_loss / len(train_loader))
    train_accuracies.append(train_acc / (BATCH_SIZE * len(train_loader)))
    model.eval()
    val_loss = 0.0
    val_acc = 0.0
    with torch.no_grad():
      for (v_batch, labels) in val_loader:
          v_batch, labels = v_batch.to(DEVICE), labels.to(DEVICE)
          labels_pred = model(v_batch)
          v_batch_loss = loss(labels_pred, labels)
          val_loss = val_loss + v_batch_loss.item()

          v_pred_max = torch.argmax(labels_pred, 1)
          batch_acc = torch.sum(v_pred_max == labels)
          val_acc = val_acc + batch_acc.item()
    val_losses.append(val_loss / len(val_loader))
    val_accuracies.append(val_acc / (BATCH_SIZE * len(val_loader)))
    print("Epoch: {}, val loss: {:.4f}, val acc: {:.4f}, train loss: {:.4f}, train acc: {:.4f},\n".format(e, val_losses[e], val_accuracies[e], train_losses[e], train_accuracies[e]))
    torch.save(model.state_dict(), SAVE_TO+"DogCat_ckpt" +  str(e)+ ".pt")



train(epochs=20)


In [ ]:
# def evaluate(model: nn.Module, loader: DataLoader) :
#   loss = nn.CrossEntropyLoss()
#   model.eval()
#   test_loss = 0.0
#   test_acc = 0.0
#   with torch.no_grad():
#     for (batch, labels) in tqdm(loader):
#       batch, labels = batch.to(DEVICE), labels.to(DEVICE)
#       y_batch_pred = model(batch)
#       batch_loss = loss(y_batch_pred, labels)
#       test_loss = test_loss + batch_loss.item()

#       pred_max = torch.argmax(y_batch_pred, 1)
#       batch_acc = torch.sum(pred_max == labels)
#       test_acc = test_acc + batch_acc.item()
#     test_loss = test_loss / len(loader)
#     test_acc = test_acc / (BATCH_SIZE * len(loader))
#     return test_loss, test_acc

def evaluate(model:  nn.Module, dataloader):
  correct = 0
  total = 0
  model.eval()
  with torch.no_grad():
      for  (batch, labels) in tqdm(dataloader):
          images, labels = batch.to(DEVICE), labels.to(DEVICE)
          outputs = model(images)
          _, predicted = torch.max(outputs.data, dim = 1)
          total += labels.size(0)
          correct += (predicted == labels).sum().item()
  return correct/total

In [ ]:
model.load_state_dict(torch.load('/content/drive/Shareddrives/cse 455/CatDogClassifer_models/DogCat_ckpt9.pt'))

test_acc = evaluate(model, test_loader)
print(f"\n Test Accuracy: {test_acc}")